# POC — Évaluation RAG avec Ragas + Langfuse
Ce notebook charge le **jeu d’or** (questions/réponses de référence), exécute votre pipeline (placeholders fournis),
calcule les métriques Ragas, exporte les scores dans l’Excel et peut tracer dans **Langfuse**.

**Prérequis** : Python 3.10+, accès au modèle (Bedrock/OpenAI/endpoint interne), et packages listés ci‑dessous.


## 0) Installation des dépendances
_Exécutez une fois. Adaptez selon votre environnement (proxy, versions)._

In [ ]:
# Si nécessaire, décommentez pour installer
# %pip install --upgrade pip
# %pip install ragas==0.2.6 langfuse==2.46.7 pydantic>=2.7.0 pandas openpyxl xlsxwriter matplotlib python-dotenv requests

## 1) Configuration
Renseignez vos variables d'environnement.
- `EVAL_INPUT_XLSX` : chemin vers le classeur Excel (celui fourni en sortie du template)
- `EVAL_OUTPUT_XLSX` : chemin de sortie (le même fichier ou un nouveau)
- `MODEL_PROVIDER` : `bedrock` | `openai` | `custom` | `ollama`
- `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_BASE_URL` si vous utilisez Langfuse
- `BATCH_SIZE` : taille du lot pour l'inférence

**Variables Ollama :**
- `OLLAMA_BASE_URL` : URL de votre instance Ollama (défaut: http://localhost:11434)
- `OLLAMA_MODEL` : nom du modèle (ex: llama2, mistral, codellama)
- `OLLAMA_TIMEOUT` : timeout en secondes
- `OLLAMA_TEMPERATURE`, `OLLAMA_TOP_P`, `OLLAMA_TOP_K` : paramètres de génération

In [ ]:
import os
import json
from dotenv import load_dotenv
load_dotenv()

# --- Paramètres principaux ---
EVAL_INPUT_XLSX = os.getenv("EVAL_INPUT_XLSX", "/mnt/data/Template_gold_POCEval.xlsx")
EVAL_OUTPUT_XLSX = os.getenv("EVAL_OUTPUT_XLSX", "/mnt/data/Resultats_POCEval.xlsx")
MODEL_PROVIDER = os.getenv("MODEL_PROVIDER", "ollama")  # bedrock | openai | custom | ollama
BATCH_SIZE = int(os.getenv("BATCH_SIZE", "16"))

# --- Langfuse ---
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY", "")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY", "")
LANGFUSE_BASE_URL  = os.getenv("LANGFUSE_BASE_URL", "https://cloud.langfuse.com")

# --- Ollama ---
OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "llama2")
OLLAMA_TIMEOUT = int(os.getenv("OLLAMA_TIMEOUT", "120"))
OLLAMA_NUM_PREDICT = int(os.getenv("OLLAMA_NUM_PREDICT", "512"))
OLLAMA_TEMPERATURE = float(os.getenv("OLLAMA_TEMPERATURE", "0.7"))
OLLAMA_TOP_P = float(os.getenv("OLLAMA_TOP_P", "0.9"))
OLLAMA_TOP_K = int(os.getenv("OLLAMA_TOP_K", "40"))

print("EVAL_INPUT_XLSX:", EVAL_INPUT_XLSX)
print("EVAL_OUTPUT_XLSX:", EVAL_OUTPUT_XLSX)
print("MODEL_PROVIDER:", MODEL_PROVIDER)
if MODEL_PROVIDER == "ollama":
    print("OLLAMA_BASE_URL:", OLLAMA_BASE_URL)
    print("OLLAMA_MODEL:", OLLAMA_MODEL)

## 2) Chargement du jeu d’or
On lit l’onglet **JEU_OR** et les **SOURCES**. On valide quelques contraintes simples pour éviter les surprises.

In [ ]:

import pandas as pd

df_or = pd.read_excel(EVAL_INPUT_XLSX, sheet_name="JEU_OR")
df_sources = pd.read_excel(EVAL_INPUT_XLSX, sheet_name="SOURCES")

required_cols = ["id","question","reponse_reference"]
missing = [c for c in required_cols if c not in df_or.columns]
if missing:
    raise ValueError(f"Colonnes manquantes dans JEU_OR: {missing}")

# Normalisation
df_or["question"] = df_or["question"].fillna("").astype(str).str.strip()
df_or["reponse_reference"] = df_or["reponse_reference"].fillna("").astype(str).str.strip()
df_or["contexte_attendu"] = df_or.get("contexte_attendu", "").fillna("").astype(str)
print("Taille du jeu d’or:", len(df_or))
df_or.head(2)


## 3) Connecteurs modèle (placeholders)
Quatre exemples : **Bedrock**, **OpenAI**, **Custom endpoint**, **Ollama**.
Remplacez les implémentations pour brancher votre vrai pipeline RAG (retrieval + génération).

In [ ]:
from typing import List, Dict, Any
import requests
import time

# --- PLACEHOLDER: récupère des contexts en se basant sur 'contexte_attendu' s'il existe.
# Remplacez par un vrai retriever (Vector DB + reranker, etc.).
def dummy_retrieve_contexts(row) -> List[str]:
    ctx = row.get("contexte_attendu", "")
    if isinstance(ctx, str) and ctx.strip():
        return [ctx]
    return []

# --- Connecteur Ollama
def call_ollama(prompt: str, max_retries: int = 3) -> Dict[str, Any]:
    """Appel à l'API Ollama avec retry et gestion d'erreur"""
    url = f"{OLLAMA_BASE_URL}/api/generate"
    payload = {
        "model": OLLAMA_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": OLLAMA_TEMPERATURE,
            "top_p": OLLAMA_TOP_P,
            "top_k": OLLAMA_TOP_K,
            "num_predict": OLLAMA_NUM_PREDICT
        }
    }
    
    for attempt in range(max_retries):
        try:
            response = requests.post(url, json=payload, timeout=OLLAMA_TIMEOUT)
            response.raise_for_status()
            result = response.json()
            return {
                "response": result.get("response", ""),
                "total_duration": result.get("total_duration", 0),
                "load_duration": result.get("load_duration", 0),
                "prompt_eval_count": result.get("prompt_eval_count", 0),
                "eval_count": result.get("eval_count", 0)
            }
        except requests.exceptions.RequestException as e:
            if attempt == max_retries - 1:
                print(f"Erreur Ollama après {max_retries} tentatives: {e}")
                return {
                    "response": "Erreur: Impossible de contacter Ollama",
                    "total_duration": 0,
                    "load_duration": 0,
                    "prompt_eval_count": 0,
                    "eval_count": 0
                }
            time.sleep(2 ** attempt)  # Backoff exponentiel
    
# --- PLACEHOLDER: génération (remplacez par vos appels Bedrock/OpenAI/SmartRAG/Ollama)
def generate_answer(question: str, contexts: List[str]) -> Dict[str, Any]:
    """Génère une réponse selon le provider configuré"""
    
    if MODEL_PROVIDER == "ollama":
        # Construction du prompt RAG
        context_text = "\n\n".join(contexts) if contexts else "Aucun contexte disponible."
        prompt = f"""Contexte:
{context_text}

Question: {question}

Réponds à la question en te basant uniquement sur le contexte fourni. Si le contexte ne contient pas l'information nécessaire, indique-le clairement.

Réponse:"""
        
        result = call_ollama(prompt)
        return {
            "answer": result["response"].strip(),
            "tokens_input": result["prompt_eval_count"],
            "tokens_output": result["eval_count"],
            "duration_ms": result["total_duration"] / 1000000,  # Conversion ns -> ms
            "model_version": f"{OLLAMA_MODEL}-ollama"
        }
    
    else:
        # Fallback pour les autres providers (placeholder)
        base = " ".join(contexts)[:800]
        if not base:
            base = "Je ne dispose pas du contexte interne. Veuillez consulter la procédure de référence."
        return {
            "answer": f"Réponse (démo) basée sur le contexte: {base}",
            "tokens_input": None,
            "tokens_output": None,
            "duration_ms": 0,
            "model_version": "demo-0.1"
        }

## 4) Inférence par batch
On produit : `reponse_modele` et `contexts_utilises` pour chaque question.
On enregistre également des métadonnées (version, config, latence si vous l’avez).

In [ ]:
from time import perf_counter
results: List[Dict[str, Any]] = []

print(f"Traitement de {len(df_or)} questions avec le provider: {MODEL_PROVIDER}")
if MODEL_PROVIDER == "ollama":
    print(f"Modèle Ollama: {OLLAMA_MODEL}")

for idx, row in df_or.iterrows():
    qid = row["id"]
    q = row["question"]
    gt = row["reponse_reference"]
    contexts = dummy_retrieve_contexts(row)

    print(f"Traitement question {idx+1}/{len(df_or)}: {qid}")
    
    t0 = perf_counter()
    generation_result = generate_answer(q, contexts)
    total_latency_ms = (perf_counter() - t0) * 1000.0
    
    # Extraction des informations selon le format retourné
    if isinstance(generation_result, dict):
        answer = generation_result.get("answer", "")
        tokens_input = generation_result.get("tokens_input")
        tokens_output = generation_result.get("tokens_output")
        model_duration_ms = generation_result.get("duration_ms", 0)
        model_version = generation_result.get("model_version", "unknown")
    else:
        # Rétrocompatibilité si generate_answer retourne une string
        answer = str(generation_result)
        tokens_input = None
        tokens_output = None
        model_duration_ms = 0
        model_version = "demo-0.1"

    results.append({
        "id": qid,
        "question": q,
        "reponse_reference": gt,
        "reponse_modele": answer,
        "contexts_utilises": contexts,
        "latence_ms": total_latency_ms,
        "latence_modele_ms": model_duration_ms,
        "tokens_entres": tokens_input,
        "tokens_sorties": tokens_output,
        "cout_estime": None,  # À calculer selon vos tarifs
        "version_modele": model_version,
        "config_retrieval": json.dumps({"provider": MODEL_PROVIDER, "k": len(contexts)}),
    })

df_pred = pd.DataFrame(results)
print(f"\nTaille des résultats: {len(df_pred)}")
print(f"Latence moyenne: {df_pred['latence_ms'].mean():.2f}ms")
if MODEL_PROVIDER == "ollama":
    print(f"Tokens moyens entrée: {df_pred['tokens_entres'].mean():.1f}")
    print(f"Tokens moyens sortie: {df_pred['tokens_sorties'].mean():.1f}")
df_pred.head(2)

## 5) Évaluation avec Ragas
On calcule les principales métriques Ragas et on agrège une synthèse.
Le `column_map` permet d’aligner nos noms de colonnes au format attendu.

In [ ]:

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entities_recall, noise_sensitivity

# Préparation du dataset au format Ragas
# Ragas attend typiquement: question(s), answer(s), contexts (list[str]), ground_truth(s)
# On mappe nos colonnes:
column_map = {
    "question": "question",
    "answer": "reponse_modele",
    "contexts": "contexts_utilises",
    "ground_truth": "reponse_reference",
}

# Ragas requiert des listes/series aux bons types
eval_df = df_pred.copy()
# Contexte doit être une liste de str
eval_df["contexts_utilises"] = eval_df["contexts_utilises"].apply(lambda x: x if isinstance(x, list) else ([] if pd.isna(x) else [str(x)]))

metrics = [faithfulness, answer_relevancy, context_precision, context_recall, context_entities_recall, noise_sensitivity]
ragas_res = evaluate(eval_df, metrics=metrics, column_map=column_map)

print("Scores par item:")
display(ragas_res.results)

print("\nMoyennes globales:")
display(ragas_res.scores)


## 6) Export des résultats vers Excel
Écrit les réponses, contextes et scores dans l’onglet `SORTIE_EVALUATIONS`. Si le fichier n’existe pas, il sera créé.

In [ ]:
# Fusion des scores item-level avec df_pred
scored = df_pred.merge(ragas_res.results, left_index=True, right_index=True, how="left")

# Colonnes de sortie (ajout de latence_modele_ms)
export_cols = [
    "id","question","reponse_reference","reponse_modele","contexts_utilises",
    "faithfulness","answer_relevancy","context_precision","context_recall",
    "context_entities_recall","noise_sensitivity","latence_ms","latence_modele_ms",
    "tokens_entres","tokens_sorties","cout_estime","version_modele","config_retrieval"
]
for c in export_cols:
    if c not in scored.columns:
        scored[c] = None

# Écriture
with pd.ExcelWriter(EVAL_OUTPUT_XLSX, engine="xlsxwriter") as writer:
    # Copier les feuilles source si besoin
    try:
        df_or.to_excel(writer, index=False, sheet_name="JEU_OR")
        df_sources.to_excel(writer, index=False, sheet_name="SOURCES")
    except Exception:
        pass
    # Écrire les scores
    scored.to_excel(writer, index=False, sheet_name="SORTIE_EVALUATIONS")

print("Export terminé:", EVAL_OUTPUT_XLSX)

## 7) (Optionnel) Traçage dans Langfuse
Cette section enregistre les traces au fil de l’eau. Décommentez et adaptez si vous utilisez Langfuse.

In [ ]:

# from langfuse import Langfuse
# if LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY:
#     lf = Langfuse(public_key=LANGFUSE_PUBLIC_KEY, secret_key=LANGFUSE_SECRET_KEY, base_url=LANGFUSE_BASE_URL)
#     for _, r in scored.iterrows():
#         trace = lf.trace(name="poc-rag-eval", input=r["question"], output=r["reponse_modele"], metadata={
#             "id": r["id"],
#             "version_modele": r["version_modele"],
#             "config_retrieval": r["config_retrieval"],
#             "scores": {
#                 "faithfulness": r.get("faithfulness"),
#                 "answer_relevancy": r.get("answer_relevancy"),
#                 "context_precision": r.get("context_precision"),
#                 "context_recall": r.get("context_recall"),
#                 "context_entities_recall": r.get("context_entities_recall"),
#                 "noise_sensitivity": r.get("noise_sensitivity"),
#             }
#         })
#     print("Traces envoyées à Langfuse.")
# else:
#     print("Langfuse non configuré (variables d’environnement manquantes).")


## 8) Synthèse & graphiques rapides

In [ ]:

import matplotlib.pyplot as plt

means = ragas_res.scores.to_dict()
labels = list(means.keys())
values = [means[k] for k in labels]

plt.figure()
plt.bar(labels, values)
plt.title("Scores moyens (Ragas)")
plt.xticks(rotation=30, ha="right")
plt.ylim(0, 1)
plt.show()


## 9) Prochaines étapes
- Remplacer les **placeholders** de retrieval/génération par votre pipeline réel (SmartRAG/Bedrock/etc.).
- Ajouter des variantes (chunking, top‑k, reranker, prompt) et **réexécuter** pour comparer.
- Utiliser Langfuse **Datasets & Runs** pour benchmarker plusieurs configs en parallèle.
- **Ollama** : Tester différents modèles locaux (llama2, mistral, codellama) en changeant `OLLAMA_MODEL`.
- Optimiser les paramètres Ollama (`temperature`, `top_p`, `top_k`) selon votre cas d'usage.